In [3]:
import requests
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Data Gathering
emissions_url = 'https://data.epa.gov/efservice/EF_W_EMISSIONS_SOURCE_GHG/rows/0:10000/JSON'
facilities_url = 'https://data.epa.gov/efservice/rlps_ghg_emitter_facilities/rows/0:10000/JSON'

emissions_data = requests.get(emissions_url).json()
facilities_data = requests.get(facilities_url).json()

emissions_df = pd.DataFrame(emissions_data)
facilities_df = pd.DataFrame(facilities_data)

merged_df = pd.merge(emissions_df, facilities_df, left_on='facility_id', right_on='facility_id')

unique_parent_companies = merged_df['parent_company'].unique()
print("Unique Parent Companies:")
for company in unique_parent_companies:
    print(company)


# Dash Initialization
app = dash.Dash(__name__)


app.layout = html.Div([
    html.H1("Methane Emissions Dashboard", style={'text-align': 'center', 'color': 'black'}),

    # Basin Selection with a dropdown
    html.Div([
        html.Label('Select Basin(s):', style={'color': 'black'}),
        dcc.Dropdown(
            id='basin-dropdown',
            options=[{'label': basin, 'value': basin} for basin in merged_df['basin_associated_with_facility'].unique()],
            multi=True,
            placeholder="Select Basin(s)",
        )
    ], style={'padding': '10px'}),

    # Slider for year range selection
    html.Div([
        html.Label('Select Year Range:', style={'color': 'black'}),
        dcc.RangeSlider(
            id='year-slider',
            min=merged_df['reporting_year'].min(),
            max=merged_df['reporting_year'].max(),
            value=[merged_df['reporting_year'].min(), merged_df['reporting_year'].max()],
            marks={str(year): str(year) for year in sorted(merged_df['reporting_year'].unique())}
        ),
    ], style={'padding': '10px'}),

    # Graphs side by side
    html.Div([
        dcc.Graph(id='emissions-year-bar', style={'width': '30%', 'display': 'inline-block'}),
        dcc.Graph(id='emissions-company-bar', style={'width': '45%', 'display': 'inline-block'}),
        dcc.Graph(id='emissions-state-map', style={'width': '25%', 'display': 'inline-block'})
    ], style={'display': 'flex', 'justify-content': 'space-between'})
    
], style={'backgroundColor': 'gray', 'padding': '20px'})



# FIRST CHART
@app.callback(
    Output('emissions-year-bar', 'figure'),
    [Input('year-slider', 'value'),
     Input('basin-dropdown', 'value')]
)
def update_year_chart(year_range, selected_basins):
    start_year, end_year = year_range

    filtered_df = merged_df[(merged_df['reporting_year'] >= start_year) & (merged_df['reporting_year'] <= end_year)]

    if selected_basins:
        filtered_df = filtered_df[filtered_df['basin_associated_with_facility'].isin(selected_basins)]

    fig = px.bar(
        filtered_df,
        x='reporting_year',
        y='total_reported_ch4_emissions',
        color='industry_segment',
        title='Methane Emissions vs. Year (Stacked by Industry Segment)',
        labels={'reporting_year': 'Year', 'total_reported_ch4_emissions': 'Methane Emissions', 'industry_segment': 'Industry Segment'}
    )
    
    fig.update_layout(
        barmode='stack',
        legend=dict(
            orientation="h",
            yanchor="top",
            y=-0.2, 
            xanchor="center",
            x=0.5,  
            tracegroupgap=10,
            font=dict(size=9)
        ),
        xaxis=dict(
            title_font=dict(size=12),
        ),
        yaxis=dict(
            title_font=dict(size=12),
        )
    )
    return fig



#SECOND CHART
@app.callback(
    Output('emissions-company-bar', 'figure'),
    [Input('year-slider', 'value')]
)
def update_company_chart(year_range):
    selected_year = year_range[1]  

    filtered_df = merged_df[(merged_df['reporting_year'] == selected_year)]

    print(filtered_df['parent_company'].unique())

    fig = px.bar(
        filtered_df,
        x='parent_company',
        y='total_reported_ch4_emissions',
        color='reporting_category',
        title=f'Methane Emissions vs. Company for {selected_year}',
        labels={'parent_company': 'Company Name', 'total_reported_ch4_emissions': 'Methane Emissions', 'reporting_category': 'Emission Source'}
    )
    company_abbreviations = {
        'PUBLIC SERVICE ENTERPRISE GROUP (100%)': 'PSEG',
        'PUBLIC SERVICE ENTERPRISE GROUP INC (100%)': 'PSEGI',
        'ARCLIGHT ENERGY PARTNERS FUND VII LP (100%)': 'ARCLIGHT',
        'SCANA CORPORATION (100%)': 'SCANA CORPO',
        'SCANA CORP (100%)': 'SCANA',
        'DOMINION ENERGY INC (100%)': 'DOMINION ENERGY',
        'CH ENERGY GROUP INC (100%)': 'CH ENERGY',
        'CENTRAL HUDSON GAS AND ELECTRIC CORPORATION (100%)': 'CENTRAL HUDSON G&E',
        'CENTRAL HUDSON GAS & ELECTRIC CORP (100%)': 'CENTRAL HUDSON G&EC',
        'CITY OF TALLAHASSEE (100%)': 'TALLAHASSEE',
        'CITY OF TALLAHASSEE FLORIDA (100%)': 'TALLAHASSEE, FL',
        'CITY OF NORWICH, CT (100%)': 'NORWICH',
        'CITY OF NORWICH CT (100%)': 'NORWICHCT',
        'CITY OF NORWICH CONNECTICUT (100%)': 'NORWICHCN',
        'SUMMIT UTILITIES, INC (100%)': 'SUMMIT',
        'OKALOOSA GAS DISTRICT (100%)': 'OKALOOSA',
        'MDU RESOURCES GROUP INC (100%)': 'MDU',
        'MIDDLEBOROUGH GAS AND  ELECTRIC DEPARTMENT (100%)': 'MIDDLEBOROUGH G&E',
        'MIDDLEBOROUGH GAS & ELECTRIC DEPARTMENT (100%)': 'MIDDLEBOROUGH G&E',
        'TOWN OF MIDDLEBOROUGH (100%)': 'MIDDLEBOROUGH',
        'TOWN OF MIDDLEBOROUGH MASSACHUSETTS (100%)': 'MIDDLEBOROUGH MS',
        'UNISOURCE ENERGY CORP (100%)': 'UNISOURCE',
        'UNS ENERGY CORP (100%)': 'UNS ENERGY',
        'PLAINS LPG SERVICES (100%)': 'PLAINS LPG',
        'PLAINS ALL AMERICAN GP LLC (100%)': 'PLAINS LPG GP',
        'PLAINS GP HOLDINGS LP (100%)': 'GP HOLDINGS',
        'ARROWHEAD PIPELINE, L.P. (100%)': 'ARROWHEAD',
        'ARROWHEAD PIPELINE, LP (100%)': 'ARROWHEAD',
        'HILCORP ENERGY CO (100%)': 'HILCORP',
        'TEXAS EASTERN TRANSMISSION, L.P. (100%)': 'TEXAS TRNSM',
        'SPECTRA ENERGY (100%)': 'SPECTRA ENERGY',
        'ENBRIDGE (US) INC (100%)': 'ENBRIDGE',
        'PIPECO LLC (100%)': 'PIPECO',
        'BROOKFIELD INFRASTRUCTURE PTNR (50%); KINDER MORGAN INC (50%)': 'KINDER',
        'BROOKFIELD INFRASTRUCTURE PARTNERS LP (50%); KINDER MORGAN INC (50%)': 'KINDERM',
        'KINDER MORGAN INC (100%)': 'KINDERMINC',
        'ALGONQUIN GAS TRANSMISSION, LLC (100%)': 'ALOGONQUIN',
        'ENBRIDGE (U.S.) INC. (50%); VERESEN INC. (50%)': 'ENBRIDGE',
        'ENBRIDGE INC (50%); VERESEN INC (50%)': 'TALLAHASSEE',
        'ENBRIDGE ENERGY MANAGEMENT (50%); VERESEN US POWER INC (50%)': 'ENBRIDGE',
        'ENBRIDGE ENERGY MANAGEMENT (50%); PEMBINA PIPELINE CORP (50%)': 'ENBRIDGE',
        'ENBRIDGE ENERGY MANAGEMENT LLC (50%); PEMBINA PIPELINE CORP (50%)': 'ENBRIDGE',
        'ALLIANCE PIPELINE LP (50%); ENBRIDGE ENERGY MANAGEMENT LLC (50%)': 'ALLIANCE',
        'ALLIANCE PIPELINE, INC (50%); ENBRIDGE (US) INC (50%)': 'ALLIENCE',
        'PACIFIC GAS AND ELECTRIC COMPANY (100%)': 'PACIFIC G&E',
        'PACIFIC GAS & ELECTRIC CO (100%)': 'PACIFICG&E',
        'PG&E CORP (100%)': 'PG&E',
    }

    abbreviated_companies = [company_abbreviations.get(company, company) for company in filtered_df['parent_company']]

    fig.update_layout(
        xaxis=dict(
            tickvals=filtered_df['parent_company'].unique(),
            ticktext=abbreviated_companies,
            title_font=dict(size=12),
            tickangle=-45
        ),
        legend=dict(
            orientation="h",
            yanchor="top",
            y=-0.2,
            xanchor="center",
            x=0.5,
            tracegroupgap=9
        )
    )
    return fig




# THIRD CHART
@app.callback(
    Output('emissions-state-map', 'figure'),
    [Input('year-slider', 'value')]
)
def update_state_map(year_range):
    selected_year = year_range[1]

    filtered_df = merged_df[(merged_df['reporting_year'] == selected_year)]

    fig = px.choropleth(
        filtered_df,
        locations='state',
        locationmode="USA-states",
        color='total_reported_ch4_emissions',
        hover_name='state',
        scope="usa",
        labels={'total_reported_ch4_emissions': 'Methane Emissions'},
        title=f'Methane Emissions by State for {selected_year}'
    )


    fig.update_layout(
        title_font=dict(size=16),  
        coloraxis_colorbar=dict(
            title='Methane Emissions',
            title_font=dict(size=10),  
            tickfont=dict(size=10), 
            orientation='h',  
            len=0.8, 
            xanchor='center',  
            x=0.5,  
            yanchor='top', 
            y=-0.1  
        ),
        width=500,
        height=500
    )

    return fig

#Running the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8080)


Unique Parent Companies:
PUBLIC SERVICE ENTERPRISE GROUP (100%)
PUBLIC SERVICE ENTERPRISE GROUP INC (100%)
ARCLIGHT ENERGY PARTNERS FUND VII LP (100%)
SCANA CORPORATION (100%)
SCANA CORP (100%)
DOMINION ENERGY INC (100%)
CH ENERGY GROUP INC (100%)
CENTRAL HUDSON GAS AND ELECTRIC CORPORATION (100%)
CENTRAL HUDSON GAS & ELECTRIC CORP (100%)
CITY OF TALLAHASSEE (100%)
CITY OF TALLAHASSEE FLORIDA (100%)
CITY OF NORWICH, CT (100%)
CITY OF NORWICH CT (100%)
CITY OF NORWICH CONNECTICUT (100%)
SUMMIT UTILITIES, INC (100%)
OKALOOSA GAS DISTRICT (100%)
MDU RESOURCES GROUP INC (100%)
MIDDLEBOROUGH GAS AND  ELECTRIC DEPARTMENT (100%)
MIDDLEBOROUGH GAS & ELECTRIC DEPARTMENT (100%)
TOWN OF MIDDLEBOROUGH (100%)
TOWN OF MIDDLEBOROUGH MASSACHUSETTS (100%)
UNISOURCE ENERGY CORP (100%)
UNS ENERGY CORP (100%)
PLAINS LPG SERVICES (100%)
PLAINS ALL AMERICAN GP LLC (100%)
PLAINS GP HOLDINGS LP (100%)
ARROWHEAD PIPELINE, L.P. (100%)
ARROWHEAD PIPELINE, LP (100%)
HILCORP ENERGY CO (100%)
TEXAS EASTERN TRANSMIS